In [1]:
import numpy as np
import pandas as pd
import scipy
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm.notebook import tqdm

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path='../../Data/Extracted/'

train_text=pd.read_csv(path+'train_multi_column_text.csv')
test_text=pd.read_csv(path+'test_multi_column_text.csv')

train_num=pd.read_csv(path+'train.csv').select_dtypes(exclude='object')
train_num.fillna(train_num.median(),inplace=True)

test_num=pd.read_csv(path+'test.csv').select_dtypes(exclude='object')
test_num.fillna(test_num.median(),inplace=True)

labels=pd.read_csv(path+'labels.csv')
labels=pd.get_dummies(labels,prefix_sep='__')

# Read submission format file
submission_format=pd.read_csv('../../Data/Original/SubmissionFormat.csv')
# Save index for submission
sub_index=(submission_format[submission_format.columns[0]].tolist())
sub_columns=submission_format.columns[1:]

del submission_format
gc.collect()

C:\Users\pushk\miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0

## 2. Tfidf vectorizer

In [3]:
train_csr=[train_num]
test_csr=[test_num]

############
### Here tfidf vectorize single column text vector if required ###
### Get the code for that from the last cell ###
############

for feat in tqdm(train_text.columns):
    vectorizer=TfidfVectorizer(ngram_range=(1,4),min_df=10)
    train_csr.append(vectorizer.fit_transform(train_text[feat].values.ravel()))
    test_csr.append(vectorizer.transform(test_text[feat].values.ravel()))

In [4]:
X_train=scipy.sparse.hstack(train_csr).tocsr()
X_test=scipy.sparse.hstack(test_csr).tocsr()

X_train.shape, X_test.shape

((382627, 9856), (50064, 9856))

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import time

In [6]:
sub_dict={}
for col in tqdm(sub_columns):
    rfc=RandomForestClassifier(n_estimators=200,
                               min_samples_leaf=10,
                               n_jobs=-1)

    rfc.fit(X_train,labels[col].values)

    sub_dict[col]=list(map(lambda val: val[1],rfc.predict_proba(X_test)))

In [7]:
sub_df=pd.DataFrame(sub_dict,columns=sub_columns,index=sub_index)

In [8]:
sub_df.to_csv('../../Data/Submissions/rfc_104_models_9.csv',index=True)

In [ ]:
"""from category_encoders.count import CountEncoder

cnt_encoder=CountEncoder()
cnt_encoder.fit(pd.concat([train_text,test_text]))
X_train=cnt_encoder.transform(train_text)
X_test=cnt_encoder.transform(test_text)


def order_by_freq(df,feat):
    '''Sort feature column according to column values in reverse order'''
    # Create a mapping dictionary where more the frequency lesser will be the rank, i.e., most frequent will be ranked 0
    sorted_list=sorted(df[feat].unique(), reverse=True)
    mapping_dict=dict(zip(list(map(lambda val:str(val),sorted_list)),range(len(sorted_list))))

    # Cast series to a string
    feat_series=df[feat].astype('str')

    # Replace column values according to a mapping dictionary
    return feat_series.replace(mapping_dict)

for df in [X_train,X_test]:
    for feat in df.columns:
        df[feat]=order_by_freq(df,feat)"""